<a href="https://colab.research.google.com/github/hahachang/colab/blob/main/price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def StartdayParseStock():
  from datetime import datetime, timedelta
  if (datetime.now()+timedelta(hours=8)).strftime("%H%M%S")<"160000":
    date   = (datetime.now()+timedelta(hours=8)+timedelta(days=-1)).strftime("%Y%m%d")  
  else:
    date   = (datetime.now()+timedelta(hours=8)).strftime("%Y%m%d")
  return date

In [ ]:
import os
import os.path
import datetime
import time
import requests
import urllib3
import pandas as pd
import json
urllib3.disable_warnings()
from time import sleep
from random import randint
from urllib import request
from datetime import datetime,timedelta #星期六日不執行的判斷。

In [ ]:
def set_loop_dates(path):
    local_filelist = os.listdir(path_docu_stock_price )

    dates_already_download = list()
    for file in local_filelist:
        x=file.split("_")[2].split(".")[0]
        dates_already_download.append(x)

    today = time.strftime("%Y%m%d", time.localtime())
    year_current = time.strftime("%Y", time.localtime())
    dates_eachday=list()
    for year in range(2004,int(year_current)+1):
        for month in range(1,13):
                for day in range(1,32):
                    dates_eachday.append(str(year*10000+month*100+day))
    dates_eachday = [i for i in dates_eachday if i <= today] 

    dates_need2download = list(set(dates_eachday)-set(dates_already_download))
    return dates_need2download

def ParseTWSE_OHLC(date):
  #date format: str and continue number
  #example    : "20210514"

  print("ParseTWSE_OHLC lanuching ..")
  from urllib import request
  import time
  import json
  def download_json(target_url):
      opener = request.build_opener()
      opener.addheaders = [('user-agent', 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36')]
      request.install_opener(opener) 
      with request.urlopen(target_url) as url:
          data = json.loads(url.read().decode())
      return data
  
  timestamp = 1000*int(time.mktime(time.localtime()))
  url_TWSE_OHLC = "https://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date="+date+"&type=ALL&_="+str(timestamp)
  json_TWSE_OHLC = download_json(url_TWSE_OHLC)
  print("ParseTWSE_OHLC closing ..")
  return json_TWSE_OHLC


def FilterOnlyStockFromTWSE_OHLC(jf):
  import pandas as pd
  #包含所有的權證及股票
  raw_dfTWSE  = pd.DataFrame(jf['data9'],columns=jf['fields9'])
  #去除comma
  dfTWSE      = raw_dfTWSE.replace(',','', regex=True) 
  dfTWSE      = dfTWSE.replace(',','', regex=True)

  #去除所有權證
  #rule1
    #認購權證之代號為6個阿拉伯數字，編號從030001至089999。
  rule1 = [str(x).zfill(6) for x in range(30001,90000)] 

  #rule2 
      #認售權證之代號為5個阿拉伯數字後加上英文字「P」，編號從03001P至08999P。
      #以外國交易所之指數或有價證券作為標的之認購權證代號為5個阿拉伯數字後加上英文字「F」，編號從03001F至08999F。
      #以外國交易所之指數或有價證券作為標的之認售權證代號為5個阿拉伯數字後加上英文字「Q」，編號從03001Q至08999Q。
      #牛證之代號為5個阿拉伯數字後加上英文字「C」，編號從03001C至08999C。
      #熊證之代號為5個阿拉伯數字後加上英文字「B」，編號從03001B至08999B。
      #展延型牛證之代號為5個阿拉伯數字後加上英文字「X」，編號從03001X至08999X。
      #展延型熊證之代號為5個阿拉伯數字後加上英文字「Y」，編號從03001Y至08999Y。    
  rule2 = "P|F|Q|C|B|X|Y"

  #apply rule1 and rule2    
  dfTWSE              = dfTWSE[~dfTWSE.證券代號.isin(rule1)] 
  dfOnlyStockFromTWSE = dfTWSE[~dfTWSE.證券代號.str.contains(rule2)]
  
  #convert data format
  dfOnlyStockFromTWSE.成交金額 = dfOnlyStockFromTWSE.成交金額.astype(float)
  return dfOnlyStockFromTWSE

def LoopsTWSE_OHLC(df):
  loops = list(df.sort_values(by="成交金額",ascending=False)["證券代號"].values)
  return loops

In [ ]:
import requests
global date
date  = StartdayParseStock()
jf    = ParseTWSE_OHLC(date)
dfOnlyStockFromTWSE = FilterOnlyStockFromTWSE_OHLC(jf)
loops = LoopsTWSE_OHLC(dfOnlyStockFromTWSE)
list_res = []
for i in loops:
  sleep(randint(2,3))
  stockcode = dfOnlyStockFromTWSE['證券代號'].values[0]
  url_deallist = 'https://tw.quote.finance.yahoo.net/quote/q?type=tick&perd=1m&mkt=10&sym='+ i +'&callback=jQuery11130000027566964186265253_1620742589816&_=1620742589817'
  res = requests.get(url_deallist)
  ress = res.content[41:len(res.content)]
  res_mod = ress.decode("utf-8").replace("(","").replace(")","").replace(";","")
  list_res.append(res_mod[5:(len(res_mod))])

  print('\r',str(list(loops).index(i)) + "/" + str(len(loops)),end='')

ParseTWSE_OHLC lanuching ..
ParseTWSE_OHLC closing ..


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


 1118/1119

In [ ]:
def SendFileToDropbox(fileLocal,fileDropbox):
    ##### upload file to dropbox #####
    try:
        import dropbox
        #print("module is installed")
    except ModuleNotFoundError:
        print("module is not installed")
        # or
        !pip install dropbox
        import dropbox

    from dropbox.files import WriteMode
    from dropbox.exceptions import ApiError, AuthError
    global tokenDropbox
    tokenDropbox = "9VjbJCirSqsAAAAAAAA1JuBzv7mujcQGbebgTGu2IjDup-N77QjVY8MEsg1ccizx"
    dbx = dropbox.Dropbox(tokenDropbox)
    with open(fileLocal, 'rb') as f:
      dbx.files_upload(f.read(), "/"+fileDropbox, mode=WriteMode('overwrite')) 
      f.close()

pd.DataFrame(list_res).to_csv("deallist_"+date+".csv",index=False)
SendFileToDropbox("deallist_"+date+".csv","deallist/"+"twse_deallist_"+date+".csv")
print("deallist to dropbox")

module is not installed
     |████████████████████████████████| 573kB 5.4MB/s 
     |████████████████████████████████| 163kB 10.2MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
deallist to dropbox


In [ ]:
list_res[0]

In [ ]:
def ParseOTC_OHLC():
  print("ParseOTC_OHLC lanuching ..")

  from urllib import request
  def download_json(target_url):
      opener = request.build_opener()
      opener.addheaders = [('user-agent', 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36')]
      request.install_opener(opener) 
      with request.urlopen(target_url) as url:
          data = json.loads(url.read().decode())
      return data
  url_otc = "https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&_=1620747726797"

  res_otc       = requests.get(url_otc)
  print(res_otc)
  json_OTC_OHLC = download_json(url_otc)
   
  return json_OTC_OHLC
  
def FilterOnlyStockFromOTC_OHLC(jf):
    raw_dfOTC =pd.DataFrame(jf['aaData'],columns=['證券代號','名稱','收盤','漲跌','開盤','最高','最低','均價','成交股數','成交金額元','成交筆數','最後買價','最後買量千股','最後賣價','最後賣量千股','發行股數','次日參考價','次日漲停價','次日跌停價'])
    #去除comma
    dfOTC = raw_dfOTC.replace(',','', regex=True) 
    dfOTC = dfOTC.replace(',','', regex=True)
        
    #去除所有權證
    #rule1
      #認購權證之代號為6個阿拉伯數字，編號從030001至089999。
    rule1 = [str(x).zfill(6) for x in range(30001,90000)] 
    #rule2
        #認售權證之代號為5個阿拉伯數字後加上英文字「P」，編號從03001P至08999P。
        #以外國交易所之指數或有價證券作為標的之認購權證代號為5個阿拉伯數字後加上英文字「F」，編號從03001F至08999F。
        #以外國交易所之指數或有價證券作為標的之認售權證代號為5個阿拉伯數字後加上英文字「Q」，編號從03001Q至08999Q。
        #牛證之代號為5個阿拉伯數字後加上英文字「C」，編號從03001C至08999C。
        #熊證之代號為5個阿拉伯數字後加上英文字「B」，編號從03001B至08999B。
        #展延型牛證之代號為5個阿拉伯數字後加上英文字「X」，編號從03001X至08999X。
        #展延型熊證之代號為5個阿拉伯數字後加上英文字「Y」，編號從03001Y至08999Y。     
    rule2 = "P|F|Q|C|B|X|Y"

   #apply rule1 and rule2 
    dfOTC = dfOTC[~dfOTC.證券代號.isin(rule1)] 
    dfOTC = dfOTC[~dfOTC.證券代號.str.contains(rule2)]
    rule3 = (dfOTC['證券代號'].str.len() == 4)
    dfOnlyStockFromOTC = dfOTC.loc[rule3]

    return dfOnlyStockFromOTC

def LoopsOTC_OHLC(df):
    df.成交金額元 = df.成交金額元.astype(float)
    loops_otc = df.sort_values(by='成交金額元',ascending=False).證券代號.values
    return loops_otc

In [ ]:
import requests
jf = ParseOTC_OHLC()
dfOnlyStockFromOTC = FilterOnlyStockFromOTC_OHLC(jf)
loops_otc          = LoopsOTC_OHLC(dfOnlyStockFromOTC)
list_res_otc = []
for i in loops_otc:
  sleep(randint(2,3))
  url_DeallistOTC = 'https://tw.quote.finance.yahoo.net/quote/q?type=tick&perd=1m&mkt=10&sym='+ i +'&callback=jQuery11130000027566964186265253_1620742589816&_=1620742589817'
  res =  requests.get(url_DeallistOTC )
  res =  res.content[41:len(res.content)]

  res_mod = ress.decode("utf-8").replace("(","").replace(")","").replace(";","")
  list_res_otc.append(res_mod[5:(len(res_mod))])
  print('\r',str(list(loops_otc).index(i)) + "/" + str(len(loops_otc)),end='')

ParseOTC_OHLC lanuching ..
<Response [200]>
 783/784

In [ ]:
pd.DataFrame(list_res_otc).to_csv("otc_deallist_"+date+".csv",index=False)
SendFileToDropbox("deallist_"+date+".csv","deallist/"+"otc_deallist_"+date+".csv")
print("otc_deallist to dropbox ..")

otc_deallist to dropbox ..


In [ ]:
list_res_otc

['{"mkt":"10","id":"020004","perd":"1m","type":"tick","mem":{"id":"020004","name":"兆豐電菁英30N","TradeDay":"20210520","129":30.46,"257":0.0,"258":0.0,"132":33.5,"133":27.42,"144":20210520,"145":0.0,"275":8.4087068E7,"156":30.08,"157":29.99,"158":30.16,"159":30.06,"288":15.133333,"160":30.14,"290":1000.0,"173":0.0,"430":31.426666,"174":0.0,"433":30.516666,"434":27.598472,"435":32.144,"436":19.0,"184":0.0,"440":29.0,"185":0.0,"443":11.0,"444":15.0,"445":31.604,"446":22.0,"448":32.357,"449":15.0,"834":32.41,"835":30.01,"836":33.86,"837":30.01,"455":30.46,"456":27.917666,"457":14.0,"209":0,"101":30.34,"102":30.35,"113":499,"114":499},"tick":[]}',
 '{"mkt":"10","id":"020004","perd":"1m","type":"tick","mem":{"id":"020004","name":"兆豐電菁英30N","TradeDay":"20210520","129":30.46,"257":0.0,"258":0.0,"132":33.5,"133":27.42,"144":20210520,"145":0.0,"275":8.4087068E7,"156":30.08,"157":29.99,"158":30.16,"159":30.06,"288":15.133333,"160":30.14,"290":1000.0,"173":0.0,"430":31.426666,"174":0.0,"433":30.51666

In [ ]:
jd = json.loads(list_res_otc[0])
jd['mem']

{'101': 30.34,
 '102': 30.35,
 '113': 499,
 '114': 499,
 '129': 30.46,
 '132': 33.5,
 '133': 27.42,
 '144': 20210520,
 '145': 0.0,
 '156': 30.08,
 '157': 29.99,
 '158': 30.16,
 '159': 30.06,
 '160': 30.14,
 '173': 0.0,
 '174': 0.0,
 '184': 0.0,
 '185': 0.0,
 '209': 0,
 '257': 0.0,
 '258': 0.0,
 '275': 84087068.0,
 '288': 15.133333,
 '290': 1000.0,
 '430': 31.426666,
 '433': 30.516666,
 '434': 27.598472,
 '435': 32.144,
 '436': 19.0,
 '440': 29.0,
 '443': 11.0,
 '444': 15.0,
 '445': 31.604,
 '446': 22.0,
 '448': 32.357,
 '449': 15.0,
 '455': 30.46,
 '456': 27.917666,
 '457': 14.0,
 '834': 32.41,
 '835': 30.01,
 '836': 33.86,
 '837': 30.01,
 'TradeDay': '20210520',
 'id': '020004',
 'name': '兆豐電菁英30N'}

In [ ]:
json.loads(jd)

TypeError: ignored

In [ ]:
jd ={"mkt":"10","id":"6488","perd":"1m","type":"tick","mem":{"id":"6488","name":"環球晶","TradeDay":"20210511","257":0.0,"258":0.0,"781":0.0,"275":8.4087068E7,"290":1000.0,"291":9.55,"292":35.08,"293":26.69,"294":18.17,"308":0.0,"822":0.0,"823":6.58,"574":0.0,"833":0.0,"323":0.0,"324":0.0,"325":0.0,"326":0.0,"327":0.0,"328":0.0,"329":0.0,"842":0.0,"843":0.0,"844":0.0,"845":0.0,"592":0.0,"849":0.0,"338":0.0,"850":0.0,"339":0.0,"340":0.0,"342":0.0,"344":0.0,"345":0.0,"101":675.0,"102":676.0,"103":674.0,"104":677.0,"105":673.0,"106":678.0,"107":672.0,"108":679.0,"109":671.0,"110":680.0,"113":10,"114":50,"115":41,"116":4,"117":24,"118":7,"119":46,"120":2,"121":20,"122":10,"125":676.0,"126":688.0,"127":676.0,"128":4860,"129":711.0,"130":697.0,"131":652.0,"132":782.0,"133":640.0,"645":3615,"646":3738,"135":0.0,"647":88,"136":0.0,"138":0.0,"396":-13.32,"397":36.53,"141":0.0,"398":28.15,"399":21.31,"143":143000,"144":20210511,"656":52200.0,"145":0.0,"401":7441000,"657":680.0,"658":679.0,"659":680.0,"404":7441,"660":2275,"150":0.0,"406":0.007441,"409":0.0,"666":20211.0,"154":0.0,"410":12000,"156":676.0,"157":631.0,"413":12,"158":720.0,"159":654.0,"160":698.0,"420":0.234318,"423":5037.722,"935":0.0,"424":50.37722,"936":0.0,"171":12,"172":6.329113,"428":0.0,"173":0.0,"174":0.0,"430":734.666666,"177":0.0,"433":799.0,"178":0.0,"434":743.402777,"179":0.0,"435":810.9,"180":0.0,"436":4464.0,"181":0.0,"182":676.0,"183":676.0,"184":-35.0,"440":6586.0,"185":-4.922644,"443":4396.0,"444":4903.0,"189":0.0,"445":765.4,"190":153.106995,"446":6579.0,"198":0.0,"199":0.0,"455":0.0,"969":675.0,"970":676.0,"971":0.0,"460":22.89,"972":0.0,"461":6.15,"973":0.0,"462":2.66,"209":7,"980":0.0,"470":1.351319,"471":677.02,"472":1.998656,"222":0.0,"229":0.0,"487":0.0,"750":1.0574592157E10,"755":4.3725E8,"501":3724},"tick":[{"t":202105110901,"p":682.00,"v":212.00},{"t":202105110902,"p":682.00,"v":78.00},{"t":202105110903,"p":683.00,"v":72.00},{"t":202105110904,"p":684.00,"v":31.00},{"t":202105110905,"p":688.00,"v":43.00},{"t":202105110906,"p":683.00,"v":56.00},{"t":202105110907,"p":681.00,"v":27.00},{"t":202105110908,"p":671.00,"v":108.00},{"t":202105110909,"p":676.00,"v":58.00},{"t":202105110910,"p":678.00,"v":38.00},{"t":202105110911,"p":677.00,"v":69.00},{"t":202105110912,"p":677.00,"v":27.00},{"t":202105110913,"p":685.00,"v":126.00},{"t":202105110914,"p":682.00,"v":3.00},{"t":202105110915,"p":686.00,"v":57.00},{"t":202105110916,"p":681.00,"v":27.00},{"t":202105110917,"p":685.00,"v":21.00},{"t":202105110918,"p":688.00,"v":45.00},{"t":202105110919,"p":687.00,"v":16.00},{"t":202105110920,"p":690.00,"v":59.00},{"t":202105110921,"p":686.00,"v":61.00},{"t":202105110922,"p":681.00,"v":24.00},{"t":202105110923,"p":680.00,"v":33.00},{"t":202105110924,"p":683.00,"v":25.00},{"t":202105110925,"p":682.00,"v":13.00},{"t":202105110926,"p":680.00,"v":19.00},{"t":202105110927,"p":682.00,"v":15.00},{"t":202105110928,"p":679.00,"v":51.00},{"t":202105110929,"p":680.00,"v":23.00},{"t":202105110930,"p":681.00,"v":16.00},{"t":202105110931,"p":675.00,"v":74.00},{"t":202105110932,"p":678.00,"v":12.00},{"t":202105110933,"p":678.00,"v":21.00},{"t":202105110934,"p":681.00,"v":80.00},{"t":202105110935,"p":680.00,"v":33.00},{"t":202105110936,"p":681.00,"v":30.00},{"t":202105110937,"p":681.00,"v":21.00},{"t":202105110938,"p":683.00,"v":34.00},{"t":202105110939,"p":686.00,"v":90.00},{"t":202105110940,"p":681.00,"v":29.00},{"t":202105110941,"p":677.00,"v":54.00},{"t":202105110942,"p":679.00,"v":17.00},{"t":202105110943,"p":676.00,"v":65.00},{"t":202105110944,"p":680.00,"v":28.00},{"t":202105110945,"p":681.00,"v":24.00},{"t":202105110946,"p":676.00,"v":37.00},{"t":202105110947,"p":675.00,"v":27.00},{"t":202105110948,"p":676.00,"v":18.00},{"t":202105110949,"p":677.00,"v":14.00},{"t":202105110950,"p":679.00,"v":13.00},{"t":202105110951,"p":677.00,"v":3.00},{"t":202105110952,"p":680.00,"v":31.00},{"t":202105110953,"p":681.00,"v":53.00},{"t":202105110954,"p":680.00,"v":5.00},{"t":202105110955,"p":681.00,"v":8.00},{"t":202105110956,"p":678.00,"v":18.00},{"t":202105110957,"p":679.00,"v":14.00},{"t":202105110958,"p":677.00,"v":7.00},{"t":202105110959,"p":667.00,"v":161.00},{"t":202105111000,"p":673.00,"v":46.00},{"t":202105111001,"p":670.00,"v":30.00},{"t":202105111002,"p":669.00,"v":35.00},{"t":202105111003,"p":665.00,"v":79.00},{"t":202105111004,"p":666.00,"v":30.00},{"t":202105111005,"p":667.00,"v":12.00},{"t":202105111006,"p":669.00,"v":37.00},{"t":202105111007,"p":670.00,"v":36.00},{"t":202105111008,"p":668.00,"v":38.00},{"t":202105111009,"p":661.00,"v":82.00},{"t":202105111010,"p":661.00,"v":38.00},{"t":202105111011,"p":658.00,"v":55.00},{"t":202105111012,"p":657.00,"v":38.00},{"t":202105111013,"p":661.00,"v":27.00},{"t":202105111014,"p":658.00,"v":33.00},{"t":202105111015,"p":656.00,"v":15.00},{"t":202105111016,"p":655.00,"v":51.00},{"t":202105111017,"p":652.00,"v":64.00},{"t":202105111018,"p":656.00,"v":22.00},{"t":202105111019,"p":658.00,"v":60.00},{"t":202105111020,"p":659.00,"v":14.00},{"t":202105111021,"p":660.00,"v":45.00},{"t":202105111022,"p":658.00,"v":24.00},{"t":202105111023,"p":657.00,"v":24.00},{"t":202105111024,"p":656.00,"v":16.00},{"t":202105111025,"p":658.00,"v":38.00},{"t":202105111026,"p":659.00,"v":17.00},{"t":202105111027,"p":660.00,"v":29.00},{"t":202105111028,"p":660.00,"v":24.00},{"t":202105111029,"p":660.00,"v":15.00},{"t":202105111030,"p":660.00,"v":8.00},{"t":202105111031,"p":660.00,"v":13.00},{"t":202105111032,"p":661.00,"v":43.00},{"t":202105111033,"p":660.00,"v":16.00},{"t":202105111034,"p":661.00,"v":3.00},{"t":202105111035,"p":663.00,"v":35.00},{"t":202105111036,"p":662.00,"v":12.00},{"t":202105111037,"p":664.00,"v":15.00},{"t":202105111038,"p":662.00,"v":41.00},{"t":202105111039,"p":664.00,"v":12.00},{"t":202105111040,"p":665.00,"v":16.00},{"t":202105111041,"p":663.00,"v":23.00},{"t":202105111042,"p":666.00,"v":14.00},{"t":202105111043,"p":667.00,"v":14.00},{"t":202105111044,"p":665.00,"v":13.00},{"t":202105111045,"p":667.00,"v":20.00},{"t":202105111046,"p":669.00,"v":21.00},{"t":202105111047,"p":670.00,"v":40.00},{"t":202105111048,"p":667.00,"v":23.00},{"t":202105111049,"p":666.00,"v":8.00},{"t":202105111050,"p":667.00,"v":10.00},{"t":202105111051,"p":668.00,"v":9.00},{"t":202105111052,"p":668.00,"v":20.00},{"t":202105111053,"p":669.00,"v":26.00},{"t":202105111054,"p":666.00,"v":18.00},{"t":202105111055,"p":668.00,"v":24.00},{"t":202105111056,"p":671.00,"v":35.00},{"t":202105111057,"p":671.00,"v":7.00},{"t":202105111058,"p":670.00,"v":6.00},{"t":202105111059,"p":675.00,"v":41.00},{"t":202105111100,"p":673.00,"v":5.00},{"t":202105111101,"p":671.00,"v":24.00},{"t":202105111102,"p":673.00,"v":15.00},{"t":202105111103,"p":672.00,"v":11.00},{"t":202105111104,"p":677.00,"v":49.00},{"t":202105111105,"p":676.00,"v":13.00},{"t":202105111106,"p":675.00,"v":11.00},{"t":202105111107,"p":674.00,"v":12.00},{"t":202105111108,"p":676.00,"v":16.00},{"t":202105111109,"p":674.00,"v":14.00},{"t":202105111110,"p":677.00,"v":28.00},{"t":202105111111,"p":678.00,"v":8.00},{"t":202105111112,"p":677.00,"v":4.00},{"t":202105111113,"p":675.00,"v":19.00},{"t":202105111114,"p":674.00,"v":10.00},{"t":202105111115,"p":677.00,"v":44.00},{"t":202105111116,"p":675.00,"v":10.00},{"t":202105111117,"p":681.00,"v":95.00},{"t":202105111118,"p":677.00,"v":22.00},{"t":202105111119,"p":678.00,"v":12.00},{"t":202105111120,"p":677.00,"v":14.00},{"t":202105111121,"p":676.00,"v":23.00},{"t":202105111122,"p":680.00,"v":24.00},{"t":202105111123,"p":681.00,"v":37.00},{"t":202105111124,"p":681.00,"v":28.00},{"t":202105111125,"p":679.00,"v":24.00},{"t":202105111126,"p":677.00,"v":20.00},{"t":202105111127,"p":681.00,"v":25.00},{"t":202105111128,"p":683.00,"v":62.00},{"t":202105111129,"p":684.00,"v":33.00},{"t":202105111130,"p":685.00,"v":25.00},{"t":202105111131,"p":680.00,"v":10.00},{"t":202105111132,"p":681.00,"v":32.00},{"t":202105111133,"p":680.00,"v":11.00},{"t":202105111134,"p":680.00,"v":14.00},{"t":202105111135,"p":682.00,"v":14.00},{"t":202105111136,"p":687.00,"v":58.00},{"t":202105111137,"p":697.00,"v":137.00},{"t":202105111138,"p":688.00,"v":26.00},{"t":202105111139,"p":686.00,"v":12.00},{"t":202105111140,"p":688.00,"v":16.00},{"t":202105111141,"p":688.00,"v":13.00},{"t":202105111142,"p":689.00,"v":12.00},{"t":202105111143,"p":686.00,"v":11.00},{"t":202105111144,"p":684.00,"v":18.00},{"t":202105111145,"p":683.00,"v":16.00},{"t":202105111146,"p":681.00,"v":32.00},{"t":202105111147,"p":685.00,"v":43.00},{"t":202105111148,"p":686.00,"v":13.00},{"t":202105111149,"p":684.00,"v":13.00},{"t":202105111150,"p":684.00,"v":8.00},{"t":202105111151,"p":682.00,"v":13.00},{"t":202105111152,"p":687.00,"v":29.00},{"t":202105111153,"p":686.00,"v":5.00},{"t":202105111154,"p":684.00,"v":28.00},{"t":202105111155,"p":686.00,"v":5.00},{"t":202105111156,"p":684.00,"v":6.00},{"t":202105111157,"p":689.00,"v":28.00},{"t":202105111158,"p":687.00,"v":3.00},{"t":202105111159,"p":685.00,"v":4.00},{"t":202105111200,"p":688.00,"v":6.00},{"t":202105111201,"p":691.00,"v":36.00},{"t":202105111202,"p":694.00,"v":43.00},{"t":202105111203,"p":690.00,"v":26.00},{"t":202105111204,"p":689.00,"v":10.00},{"t":202105111205,"p":689.00,"v":5.00},{"t":202105111206,"p":690.00,"v":7.00},{"t":202105111207,"p":688.00,"v":4.00},{"t":202105111208,"p":688.00,"v":8.00},{"t":202105111209,"p":687.00,"v":10.00},{"t":202105111210,"p":690.00,"v":6.00},{"t":202105111211,"p":688.00,"v":3.00},{"t":202105111212,"p":688.00,"v":11.00},{"t":202105111213,"p":687.00,"v":14.00},{"t":202105111214,"p":686.00,"v":3.00},{"t":202105111215,"p":687.00,"v":26.00},{"t":202105111216,"p":688.00,"v":3.00},{"t":202105111217,"p":685.00,"v":22.00},{"t":202105111218,"p":690.00,"v":27.00},{"t":202105111219,"p":690.00,"v":9.00},{"t":202105111220,"p":689.00,"v":4.00},{"t":202105111221,"p":686.00,"v":11.00},{"t":202105111222,"p":684.00,"v":22.00},{"t":202105111223,"p":686.00,"v":9.00},{"t":202105111224,"p":685.00,"v":9.00},{"t":202105111225,"p":687.00,"v":6.00},{"t":202105111226,"p":685.00,"v":6.00},{"t":202105111227,"p":686.00,"v":13.00},{"t":202105111228,"p":685.00,"v":3.00},{"t":202105111229,"p":685.00,"v":27.00},{"t":202105111230,"p":688.00,"v":48.00},{"t":202105111231,"p":689.00,"v":27.00},{"t":202105111232,"p":690.00,"v":18.00},{"t":202105111233,"p":688.00,"v":5.00},{"t":202105111234,"p":689.00,"v":10.00},{"t":202105111235,"p":686.00,"v":33.00},{"t":202105111236,"p":691.00,"v":27.00},{"t":202105111237,"p":692.00,"v":20.00},{"t":202105111238,"p":691.00,"v":4.00},{"t":202105111239,"p":689.00,"v":13.00},{"t":202105111240,"p":689.00,"v":5.00},{"t":202105111241,"p":688.00,"v":7.00},{"t":202105111242,"p":689.00,"v":5.00},{"t":202105111243,"p":687.00,"v":8.00},{"t":202105111244,"p":685.00,"v":32.00},{"t":202105111245,"p":687.00,"v":23.00},{"t":202105111246,"p":685.00,"v":5.00},{"t":202105111247,"p":684.00,"v":8.00},{"t":202105111248,"p":683.00,"v":26.00},{"t":202105111249,"p":681.00,"v":38.00},{"t":202105111250,"p":681.00,"v":24.00},{"t":202105111251,"p":683.00,"v":13.00},{"t":202105111252,"p":686.00,"v":10.00},{"t":202105111253,"p":680.00,"v":36.00},{"t":202105111254,"p":681.00,"v":22.00},{"t":202105111255,"p":680.00,"v":10.00},{"t":202105111256,"p":681.00,"v":9.00},{"t":202105111257,"p":678.00,"v":37.00},{"t":202105111258,"p":677.00,"v":51.00},{"t":202105111259,"p":681.00,"v":7.00},{"t":202105111300,"p":678.00,"v":12.00},{"t":202105111301,"p":677.00,"v":11.00},{"t":202105111302,"p":676.00,"v":33.00},{"t":202105111303,"p":675.00,"v":31.00},{"t":202105111304,"p":678.00,"v":26.00},{"t":202105111305,"p":679.00,"v":3.00},{"t":202105111306,"p":681.00,"v":24.00},{"t":202105111307,"p":682.00,"v":32.00},{"t":202105111308,"p":675.00,"v":84.00},{"t":202105111309,"p":675.00,"v":42.00},{"t":202105111310,"p":673.00,"v":64.00},{"t":202105111311,"p":671.00,"v":24.00},{"t":202105111312,"p":678.00,"v":64.00},{"t":202105111313,"p":676.00,"v":6.00},{"t":202105111314,"p":679.00,"v":14.00},{"t":202105111315,"p":677.00,"v":6.00},{"t":202105111316,"p":676.00,"v":14.00},{"t":202105111317,"p":675.00,"v":26.00},{"t":202105111318,"p":674.00,"v":16.00},{"t":202105111319,"p":673.00,"v":22.00},{"t":202105111320,"p":676.00,"v":13.00},{"t":202105111321,"p":680.00,"v":31.00},{"t":202105111322,"p":679.00,"v":6.00},{"t":202105111323,"p":677.00,"v":31.00},{"t":202105111324,"p":677.00,"v":23.00},{"t":202105111325,"p":675.00,"v":32.00},{"t":202105111330,"p":676.00,"v":257.00}]}

In [ ]:
jd.keys()
jd['tick']